In [2]:
#spark

In [3]:
! rm -rf metastore_db/

In [4]:
#spark.stop()

In [5]:
spark = SparkSession \
    .builder \
    .appName("Prediction of Loan Payment 20 Feb 2020") \
    .master("local[*]") \
    .config("spark.executor.cores","3")\
    .getOrCreate();

# 1. Data Understanding

## Collecting the current data from the LendingClub's Website.

In [6]:
#! rm -rf LoanStats_web.csv

In [7]:
#! touch LoanStats_web.csv

In [8]:
! ls -l /root/notebook/data/LoanStats_web.csv

-rw-r--r-- 1 root root 1160243241 Nov 22 18:37 /root/notebook/data/LoanStats_web.csv


In [9]:
! head -3 /root/notebook/data/LoanStats_web.csv

"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","open_il_12m","open_il_24m","mths_since_rcnt_il","total_bal_il","il_util","open_rv_1

In [ ]:
! wc -l /root/notebook/data/LoanStats_web.csv

1432493 /root/notebook/data/LoanStats_web.csv


In [ ]:
raw_df = spark.read.format('csv').option('header','true').option('mode','DROPMALFORMED')\
.load('/root/notebook/data/LoanStats_web.csv')

In [ ]:
raw201617_df = raw_df

In [ ]:
raw201617_df.printSchema()

### Numbers of Fields [1]

In [ ]:
len(raw201617_df.columns)

In [ ]:
rawweb_df = raw201617_df.select('id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 #'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m')

### Numbers of Fields after attributes selections [1]

In [ ]:
len(rawweb_df.columns)

In [ ]:
rawweb_df.select(['issue_d']).distinct().show()

In [ ]:
raw_df = rawweb_df

In [ ]:
raw_df.count()

In [ ]:
df_colfam5 = raw_df.select("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d")

### Numbers of Fields after final attributes selections [1]

In [ ]:
len(df_colfam5.columns)

In [ ]:
! pip install pandas

In [ ]:
import pandas as pd
pd.DataFrame(df_colfam5.describe().take(6), columns=df_colfam5.describe().columns).transpose()

df_colfam5.select(['last_credit_pull_d']).distinct().\
orderBy(['last_credit_pull_d'],ascending=1).show()

df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5.last_credit_pull_d == '').count()

df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5['last_credit_pull_d'].isNull()).count()

In [ ]:
df_colfam5.groupby('home_ownership').count().show()

# 2. Data Preparation

Divide this process into 2 parts. The first is a business oriented preparation that turn many business rules to be programming's logics. Its result benefits many tasks related to the Business Intelligence and other descriptive analytics. The second one is a data science oriented preparation that turn many requirements of data science to be programming's logics. Its result benefits many tasks related to the Predictive analytics.

Asumption: Business oriented preparation: No missing values, ONLY month, Correct data types.

## Data Cleansing: Remove missing values

In [ ]:
df_no_missing = df_colfam5.dropna(how='any')

## Check for a schema

In [ ]:
df_no_missing.printSchema()

In [ ]:
df_no_missing_fitmem = df_no_missing.repartition(60)

In [ ]:
df_no_missing_cached = df_no_missing_fitmem.cache()

In [ ]:
df_no_missing_cached.registerTempTable("df")

## Number of data rows

In [ ]:
df_no_missing_cached.count()

## Data Transformation: Remove Sign of Percent and Extract Month.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

### Remove Sign of Percent

In [ ]:
def f_removepercentsign(origin):
    return origin.rstrip('%')

In [ ]:
removepercentsign = udf(lambda x: f_removepercentsign(x),StringType())

### Extract Month

In [ ]:
def f_extractmonth(origin):
    return origin.split('-')[0]

In [ ]:
extractmonth = udf(lambda x: f_extractmonth(x),StringType())

In [ ]:
extractterm = udf(lambda x: x.replace('months',''),StringType())

In [ ]:
#extractemp_length = udf(lambda x: x.replace('year',''),StringType())

In [ ]:
from pyspark.sql.functions import col

In [ ]:
dfWithCrunch = df_no_missing_cached.\
withColumn('revol_util',removepercentsign(col('revol_util')).cast(DoubleType())).\
withColumn('int_rate',removepercentsign(col('int_rate')).cast(DoubleType())).\
withColumn('earliest_cr_line',extractmonth(col('earliest_cr_line')).cast(StringType())).\
withColumn('last_credit_pull_d',extractmonth(col('last_credit_pull_d')).cast(StringType())).\
withColumn('dti',col('dti').cast(DoubleType())).\
withColumn('loan_amnt',col('loan_amnt').cast(DoubleType())).\
withColumn('revol_bal',col('revol_bal').cast(DoubleType())).\
withColumn('term',extractterm(col('term')).cast(DoubleType())).\
withColumn('installment',col('installment').cast(DoubleType())).\
withColumn('open_acc',col('open_acc').cast(DoubleType())).\
withColumn('total_acc',col('total_acc').cast(DoubleType())).\
withColumn('pub_rec',col('pub_rec').cast(DoubleType())).\
withColumn('annual_inc',col('annual_inc').cast(DoubleType()))

In [ ]:
#dfWithCrunch.select(['last_credit_pull_d']).printSchema()

In [ ]:
#dfWithCrunch.groupby('emp_length').count().show()

In [ ]:
#dfWithCrunch.count()

In [ ]:
dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade','term']).show()

In [ ]:
#dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

In [ ]:
dfWithCrunch.select(['loan_amnt','annual_inc']).describe().show()

In [ ]:
rawhive_df = dfWithCrunch.repartition(60).cache()
#rawhive_df.registerTempTable("crunched_data")

In [ ]:
len(rawhive_df.columns)

In [ ]:
rawhive_df.printSchema()

In [ ]:
#rawhive_df.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

In [ ]:
rawhive_df.count()

### Data Transformation: Normalization of "annual_inc" and "loan_amnt"

In [ ]:
from pyspark.sql.functions import *

max_annual_inc = rawhive_df.select(max('annual_inc')).collect()[0][0]

min_annual_inc = rawhive_df.select(min('annual_inc')).collect()[0][0]

#sqlContext.udf.register("t_annual_inc", lambda x: ((x-min_annual_inc)/(max_annual_inc-min_annual_inc)))

In [ ]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [ ]:
normalized_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

In [ ]:
max_loan_amnt = rawhive_df.select(max('loan_amnt')).collect()[0][0]

min_loan_amnt = rawhive_df.select(min('loan_amnt')).collect()[0][0]

#sqlContext.udf.register("t_loan_amnt", lambda x: ((x-min_loan_amnt)/(max_loan_amnt-min_loan_amnt)))

In [ ]:
def t_loan_amnt(origin):
    return ((origin-min_loan_amnt)/(max_loan_amnt-min_loan_amnt))

In [ ]:
normalized_loan_amnt = udf(lambda x: t_loan_amnt(x),DoubleType())

In [ ]:
normalized_df = rawhive_df.withColumn('loan_amnt',normalized_loan_amnt(col('loan_amnt'))).\
withColumn('annual_inc',normalized_annual_inc(col('annual_inc')))

In [ ]:
normalized_df.printSchema()

### Number of data rows that are only "Fully Paid" and "Charged Off"

In [ ]:
from pyspark.sql.functions import col

In [ ]:
normalized_df.select(col('loan_status')).groupBy('loan_status').count().show()

In [ ]:
normalized_df.select(['loan_amnt','term','int_rate','installment','grade','annual_inc','revol_bal','loan_status']).\
orderBy(['loan_amnt','term','int_rate','installment','grade','annual_inc']).show()

In [ ]:
normalized_df.select(['loan_amnt','annual_inc']).describe().show()

In [ ]:
normalized_filtered_df = normalized_df.filter(col('loan_status') == 'Fully Paid').\
union(normalized_df.filter(col('loan_status') == 'Charged Off'))

In [ ]:
data = normalized_filtered_df.repartition(60).cache()

### Drop Null

In [ ]:
data_no_missing_df = data.dropna(how='any')

In [ ]:
data_no_missing_df.count()

In [ ]:
df_no_missing_cached.select(['loan_status']).distinct().show(100)

In [ ]:
data_no_missing_df.printSchema()

# 3. Data Modeling

In [ ]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder,\
VectorIndexer, QuantileDiscretizer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, NaiveBayes, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [ ]:
installmentDiscretizer = QuantileDiscretizer(numBuckets=5,inputCol='installment',\
                                             outputCol='installmentDiscreted')

In [ ]:
labelIndexer = StringIndexer(inputCol='loan_status',outputCol='indexedLabel')

In [ ]:
homeIndexer = StringIndexer(inputCol='home_ownership',outputCol='homeIndexed')
homeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='homeIndexed',outputCol='homeVec')

In [ ]:
gradeIndexer = StringIndexer(inputCol='grade',outputCol='gradeIndexed')
gradeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='gradeIndexed',outputCol='gradeVec')

In [ ]:
purposeIndexer = StringIndexer(inputCol='purpose',outputCol='purposeIndexed')
purposeOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='purposeIndexed',\
                                     outputCol='purposeVec')

In [ ]:
emp_lengthIndexer = StringIndexer(inputCol='emp_length',outputCol='emp_lengthIndexed')
emp_lengthOneHotEncoder = OneHotEncoder(dropLast=False,inputCol='emp_lengthIndexed',\
                                     outputCol='emp_lengthVec')

In [ ]:
featureAssembler = VectorAssembler(inputCols=[\
                                              'loan_amnt',\
                                              'int_rate',\
                                              #'revol_bal',\
                                              'annual_inc',\
                                              'emp_lengthVec',\
                                              'installmentDiscreted',\
                                              'dti',\
                                              'homeVec',\
                                              'gradeVec',\
                                              'open_acc',\
                                              'term'],\
                                   outputCol='features')

In [ ]:
data_no_missing_df_fully_paid = data_no_missing_df.filter(col('loan_status') == 'Fully Paid').sample(True, 0.3, 42)

In [ ]:
data_no_missing_df_charge_off = data_no_missing_df.filter(col('loan_status') == 'Charged Off')

In [ ]:
final_data_no_missing_df = data_no_missing_df_fully_paid.union(data_no_missing_df_charge_off)

In [ ]:
training , test = final_data_no_missing_df.filter(col('loan_amnt') > 0).randomSplit([0.8,0.2])

In [ ]:
training.select('loan_amnt').describe().show()

In [ ]:
training.select(['revol_bal']).\
describe().show()

In [ ]:
training.select(['loan_amnt','term','int_rate','installment','annual_inc']).\
describe().show()

In [ ]:
training

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

pipeline = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    #home_ownershipIndexer,home_ownershipOneHotEncoder,\
                                    installmentDiscretizer,\
                                    labelIndexer,featureAssembler])

In [ ]:
train_df_features = pipeline.fit(training).transform(training)

### MLP: กำหนด Pipeline เพื่อ Train Network

In [ ]:
layers = [
    train_df_features.schema["features"].metadata["ml_attr"]["num_attrs"],
    20, 10, 2
]

In [ ]:
layers

In [ ]:
#layers = [30, 5, 4, 2]

clf = MultilayerPerceptronClassifier(labelCol='indexedLabel',featuresCol='features'\
                                     ,layers=layers)

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier

pipeline = Pipeline(stages=[gradeIndexer,gradeOneHotEncoder,\
                                    homeIndexer,homeOneHotEncoder,\
                                    purposeIndexer,purposeOneHotEncoder,\
                                    emp_lengthIndexer,emp_lengthOneHotEncoder,\
                                    #home_ownershipIndexer,home_ownershipOneHotEncoder,\
                                    installmentDiscretizer,\
                                    labelIndexer,featureAssembler,clf])

In [ ]:
#getattr(clf,x1)

In [ ]:
#getattr(clf,x2)

In [ ]:
x1 = 'stepSize'
x2 = 'maxIter'
paramGrid = ParamGridBuilder() \
    .addGrid(getattr(clf,x1), [0.1, 0.2]) \
    .addGrid(getattr(clf,x2),[5,10])\
    .build()
evaluator = MulticlassClassificationEvaluator\
(labelCol='indexedLabel', predictionCol='prediction', metricName='f1')

crossval = CrossValidator(estimator=pipeline,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=evaluator,
                                      numFolds=2)
cvModel = crossval.fit(training)

In [ ]:
cvModel

model1 = pipeline_dt.fit(training)

In [ ]:
training.select(['loan_amnt','int_rate','dti','home_ownership','grade']).describe().show()

In [ ]:
training.select(['loan_amnt','int_rate','dti','home_ownership','grade']).show()

# 4. Evaluation of Models

In [ ]:
raw_result = cvModel.transform(test)

In [ ]:
spark_labeling = udf(lambda x: "Charged Off" if x == 1.0 else "Fully Paid")

In [ ]:
result = raw_result.withColumn('prediction_label',spark_labeling(col('prediction')))

In [ ]:
test.count()

In [ ]:
training.count()

In [ ]:
training.groupBy('loan_status').count().show()

In [ ]:
(0.25*349664)

In [ ]:
training.filter(training.loan_status == 'Fully Paid').count()

In [ ]:
training.filter(training.loan_status == 'Charged Off').count()

In [ ]:
result.select('loan_status','indexedLabel','prediction','prediction_label')\
.filter(result.loan_status != 'Fully Paid').show(10)

In [ ]:
result.select('loan_status','indexedLabel','prediction')\
.filter(result.loan_status != 'Fully Paid').count()

In [ ]:
result.select('loan_status','indexedLabel','prediction')\
.filter(result.loan_status != 'Fully Paid').\
filter(result.indexedLabel == result.prediction).count()

In [ ]:
result.select('loan_status','indexedLabel','prediction')\
.filter(result.loan_status != 'Fully Paid').\
filter(result.indexedLabel != result.prediction).count()

In [ ]:
result.select('loan_status','indexedLabel','prediction','prediction_label')\
.filter(result.loan_status == 'Fully Paid').show(20)

In [ ]:
result.select('loan_status','indexedLabel','prediction','prediction_label')\
.filter(result.loan_status != 'Fully Paid').show(20)

In [ ]:
result.select('loan_status','indexedLabel','prediction')\
.filter(result.indexedLabel != result.prediction).\
filter(result.loan_status == 'Fully Paid').count()

In [ ]:
result.select('loan_status','indexedLabel','prediction')\
.filter(result.indexedLabel != result.prediction)\
.filter(result.loan_status == 'Charged Off').count()

## Calculation for Confusion Matrix.

In [ ]:
def eval_metrics(lap):

    tp = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Charged Off')]))
    
    fn = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Fully Paid')]))

    tn = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Fully Paid')]))

    fp = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Charged Off')]))


    try:
        positivepredictivevalue = tp / (tp+fp)
        negativepredictivevalue = tn / (tn+fn)
        sensitivity = tp / (tp+fn)
        specificity = tn / (tn+fp)
        accuracy = (tp+tn) / (tp+tn+fp+fn)
        return {'PPV': positivepredictivevalue, 'Sensitivity': sensitivity,\
            'NPV':negativepredictivevalue, 'Specificity': specificity, 'Accuracy': accuracy}
    except ZeroDivisionError:
        return 0

    

In [ ]:
! pip install pandas

In [ ]:
lap = result.select("loan_status", "prediction_label").toPandas()

In [ ]:
#lap

In [ ]:
m = eval_metrics(lap)

In [ ]:
m

In [ ]:
tp = float(len(lap[(lap['loan_status']=='Charged Off') &\
                   (lap['prediction_label']=='Charged Off')])) #positive = Charged Off

In [ ]:
tp

In [ ]:
tn = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                   (lap['prediction_label']=='Fully Paid')])) #negative = Fully Paid

In [ ]:
tn

In [ ]:
fp = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                   (lap['prediction_label']=='Charged Off')]))

In [ ]:
fn = float(len(lap[(lap['loan_status']=='Charged Off') &\
                   (lap['prediction_label']=='Fully Paid')]))

In [ ]:
#! pip install plotly

In [ ]:
#! pip install chart-studio

In [ ]:
#model1.save('./model_24Nov2019/')

In [ ]:
#model1

In [ ]:
#! ls -l ./model_24Nov2019/stages/

In [ ]:
#m

# Jan2020

In [ ]:
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import Bucketizer

In [ ]:
def get_model(df,categoricalCols,continuousCols,\
              discretedCols,split_range,labelCol):

    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    from pyspark.sql.functions import col
    
    
    labelIndexer = StringIndexer(inputCol=labelCol,\
                             outputCol='indexedLabel',\
                             handleInvalid='keep')

    indexers = [ StringIndexer(handleInvalid='keep',\
                               inputCol=c, outputCol="{0}_indexed".format(c))
                 for c in categoricalCols ]

    # default setting: dropLast=True
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(),
                 outputCol="{0}_encoded".format(indexer.getOutputCol()))
                 for indexer in indexers ]
    discretizers = [ Bucketizer(inputCol=d, outputCol="{0}_discretized".format(d)\
                 ,splits=split_range)
                 for d in discretedCols ]
    
    
    featureCols = ['features']
    assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders]
                                + continuousCols +\
                                [discretizer.getOutputCol() for discretizer in discretizers], \
                                outputCol='features')
    

    pipeline_feature = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler])
    
    
    
    train_df_features = pipeline_feature.fit(training).transform(training)
    
    layers = [
    train_df_features.schema["features"].metadata["ml_attr"]["num_attrs"],
    20, 10, 5, 2]
    
    
    clf = MultilayerPerceptronClassifier(labelCol='indexedLabel',featuresCol='features'\
                                     ,layers=layers)
    
    
    pipeline = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler] + [clf])
    


    model=pipeline.fit(df)


    return model

In [ ]:
training

In [ ]:
training.select('grade').describe().show()

In [ ]:
catcols = [\
           'emp_length',\
           'home_ownership',\
           'verification_status',\
           #'purpose',\
           'grade',\
           'addr_state'\
          ]

num_cols = [\
            #'dti',\
            'loan_amnt',\
            'int_rate',\
            'installment',\
            'annual_inc',\
            #'revol_bal',\
            #'delinq_2yrs',\
            'pub_rec',\
            #'revol_util'\
            'total_acc'\
           ]

discols = [\
           #'pub_rec',\
           'dti',\
           #'installment'\
          ]



labelCol = 'loan_status'

splits = [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, float("inf")]

In [ ]:
from itertools import combinations 
list(combinations(catcols, 2))

In [ ]:
for i in range(len(catcols)):
    for j in (list(combinations(catcols, i+1))):
        ml_model = get_model(training,j\
                             ,num_cols,discols, splits, labelCol)
        lap2 = ml_model.transform(test)\
        .withColumn('prediction_label',spark_labeling(col('prediction')))\
        .toPandas()
        m2 = eval_metrics(lap2)
        print("Categorical Features: {0} ได้ Model ที่มี {1}".format(j,m2))
        

In [ ]:
#ml_model = get_model(training,catcols,num_cols,discols, splits, labelCol)

In [ ]:
#ml_model

In [ ]:
ml_model.stages

In [ ]:
get_bucket = ml_model.stages[11]

In [ ]:
get_bucket.getOutputCol()

In [ ]:
get_bucket.transform(training).groupBy(get_bucket.getOutputCol()).count().show()

In [ ]:
lap2 = ml_model.transform(test)\
.withColumn('prediction_label',spark_labeling(col('prediction')))\
.toPandas()

In [ ]:
m2 = eval_metrics(lap2)

In [ ]:
m2

In [ ]:
training.groupby('loan_status').count().show()

In [ ]:
test.groupby('loan_status').count().show()

In [ ]:
training.count()

In [ ]:
training.count()

In [ ]:
training.groupby('loan_status').count().show()

In [ ]:
test.groupBy('loan_status').count().show()

In [ ]:
test_result_df = ml_model.transform(test)

In [ ]:
test_result_df.filter(col('indexedLabel') != col('prediction')).count()

In [ ]:
test_result_df.filter(col('indexedLabel') != col('prediction')).filter(col('loan_status') == 'Charged Off').\
groupBy(['loan_status','indexedLabel','prediction']).count().show()

In [ ]:
test_result_df.filter(col('loan_status') == 'Charged Off').count()

In [ ]:
13903/30629